<a href="https://colab.research.google.com/github/anastasiiaf/ibm_data_engineering/blob/main/ExtractTransformLoad_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Extract Transform Load (ETL) Lab**


Estimated time needed: **30** minutes


## Objectives

After completing this lab you will be able to:

*   Read CSV and JSON file types.
*   Extract data from the above file types.
*   Transform data.
*   Save the transformed data in a ready-to-load format which data engineers can use to load into an RDBMS.


Import the required modules and functions


In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

## Download Files


In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2022-03-18 21:47:59--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: ‘source.zip.2’

source.zip.2        100%[===================>]   2.64K  --.-KB/s    in 0s      

2022-03-18 21:47:59 (327 MB/s) - ‘source.zip.2’ saved [2707/2707]



## Unzip Files


In [3]:
!unzip source.zip

Archive:  source.zip
replace source3.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: source3.json            
replace source1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: source1.csv             
replace source2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: source2.csv             
replace source3.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: source3.csv             
replace source1.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: source1.json            
replace source2.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: source2.json            
replace source1.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: source1.xml             
replace source2.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: source2.xml             
replace source3.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: source3.xml             


## Set Paths


In [4]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

## Extract


### CSV Extract Function


In [5]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

### JSON Extract Function


In [6]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

### XML Extract Function


In [7]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index=True)
    return dataframe

### Extract Function


In [8]:
def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

## Transform


The transform function does the following tasks.

1.  Convert height which is in inches to millimeter
2.  Convert weight which is in pounds to kilograms


In [9]:
def transform(data):
        #Convert height which is in inches to millimeter
        #Convert the datatype of the column into float
        #data.height = data.height.astype(float)
        #Convert inches to meters and round off to two decimals(one inch is 0.0254 meters)
        data['height'] = round(data.height * 0.0254,2)
        
        #Convert weight which is in pounds to kilograms
        #Convert the datatype of the column into float
        #data.weight = data.weight.astype(float)
        #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
        data['weight'] = round(data.weight * 0.45359237,2)
        return data

## Loading


In [10]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

## Logging


In [11]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running ETL Process


In [12]:
log("ETL Job Started")

In [13]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,name,height,weight,Unnamed: 0,car_model,year_of_manufacture,price,fuel
0,alex,65.78,112.99,NaN,NaN,NaN,NaN,NaN
1,ajay,71.52,136.49,NaN,NaN,NaN,NaN,NaN
2,alice,69.40,153.03,NaN,NaN,NaN,NaN,NaN
3,ravi,68.22,142.34,NaN,NaN,NaN,NaN,NaN
4,joe,67.79,144.30,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
163,ivan,67.62,114.14,NaN,NaN,NaN,NaN,NaN
164,simon,67.90,112.37,NaN,NaN,NaN,NaN,NaN
165,jacob,66.78,120.67,NaN,NaN,NaN,NaN,NaN
166,cindy,66.49,127.45,NaN,NaN,NaN,NaN,NaN


In [14]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,name,height,weight,Unnamed: 0,car_model,year_of_manufacture,price,fuel
0,alex,1.67,51.25,NaN,NaN,NaN,NaN,NaN
1,ajay,1.82,61.91,NaN,NaN,NaN,NaN,NaN
2,alice,1.76,69.41,NaN,NaN,NaN,NaN,NaN
3,ravi,1.73,64.56,NaN,NaN,NaN,NaN,NaN
4,joe,1.72,65.45,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
163,ivan,1.72,51.77,NaN,NaN,NaN,NaN,NaN
164,simon,1.72,50.97,NaN,NaN,NaN,NaN,NaN
165,jacob,1.70,54.73,NaN,NaN,NaN,NaN,NaN
166,cindy,1.69,57.81,NaN,NaN,NaN,NaN,NaN


In [15]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [16]:
log("ETL Job Ended")

# Exercise


Using the example above complete the exercise below.


## Download Files


In [17]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2022-03-18 21:49:44--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip.1’

datasource.zip.1    100%[===================>]   4.15K  --.-KB/s    in 0s      

2022-03-18 21:49:45 (313 MB/s) - ‘datasource.zip.1’ saved [4249/4249]



## Unzip Files


In [18]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
replace dealership_data/used_car_prices1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dealership_data/used_car_prices1.csv  
replace dealership_data/used_car_prices2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dealership_data/used_car_prices2.csv  
replace dealership_data/used_car_prices3.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dealership_data/used_car_prices3.csv  
replace dealership_data/used_car_prices1.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dealership_data/used_car_prices1.json  
replace dealership_data/used_car_prices2.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dealership_data/used_car_prices2.json  
replace dealership_data/used_car_prices3.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dealership_data/used_car_prices3.json  
replace dealership_data/used_car_prices1.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dealership_data/used_car_prices1.xml  
rep

## About the Data


The file `dealership_data` contains CSV, JSON, and XML files for used car data which contain features named `car_model`, `year_of_manufacture`, `price`, and `fuel`.


## Set Paths


In [19]:
tmpfile2    = "dealership_temp.tmp"               # file used to store all extracted data
logfile2    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile2 = "dealership_transformed_data.csv"   # file where transformed data is stored

## Extract


### Question 1: CSV Extract Function


In [20]:
# Add the CSV extract function below
def extract_from_csv2(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

<details><summary>Click here for the solution</summary>

```
    
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe
```

</details>


### Question 2: JSON Extract Function


In [21]:
# Add the JSON extract function below
def extract_from_json2(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

<details><summary>Click here for the solution</summary>

```
    
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe
```

</details>


### Question 3: XML Extract Function


In [22]:
# Add the XML extract function below, it is the same as the xml extract function above but the column names need to be renamed.
def extract_from_xml2(file_to_process):
    dataframe = pd.DataFrame(columns=["car_model", "year_of_manufacture", "price", "fuel"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for car in root:
        car_model = car.find("car_model").text
        year_of_manufacture = int(car.find("year_of_manufacture").text)
        price = float(car.find("price").text)
        fuel = car.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe

<details><summary>Click here for the solution</summary>

```
    
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe
```

</details>


### Question 4: Extract Function

Call the specific extract functions you created above by replacing the `ADD_FUNCTION_CALL` with the proper function call.


In [23]:
def extract2():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv2(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json2(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml2(xmlfile), ignore_index=True)
        
    return extracted_data

<details><summary>Click here for the solution</summary>

```
    
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data
```

</details>


## Transform


### Question 5: Transform

Round the `price` columns to 2 decimal places


In [24]:
# Add the transform function below
def transform2(data):
        data['price'] = round(data.price,2)
        
        return data

<details><summary>Click here for the solution</summary>

```

def transform(data):
        data['price'] = round(data.price, 2)
        return data
```

</details>


## Loading


### Question 6: Load


In [25]:
# Add the load function below
def load2(targetfile2,data_to_load2):
    data_to_load2.to_csv(targetfile2) 

<details><summary>Click here for the solution</summary>

```

def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  
```

</details>


## Logging


### Question 7: Log

Make sure to change the name of the logfile to the one specified in the set paths section. Change the timestamp order to Hour-Minute-Second-Monthname-Day-Year.


In [26]:
# Add the log function below
def log2(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

<details><summary>Click here for the solution</summary>

```

def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n') 
```

</details>


## Running ETL Process


### Question 8: ETL Process

Run all functions to extract, transform, and load the data. Make sure to log all events using the `log` function. Place your code under each comment.


In [27]:
# Log that you have started the ETL process
log2("ETL Job Started")

# Log that you have started the Extract step
log2("Extract phase Started")

# Call the Extract function
extracted_data2 = extract2()
# Log that you have completed the Extract step
log2("Extract phase Ended")
extracted_data2



,car_model,year_of_manufacture,price,fuel
0,alto 800,2017,4253.731343,Petrol
1,ciaz,2015,10223.880597,Diesel
2,ciaz,2015,11194.029851,Petrol
3,ertiga,2015,9104.477612,Petrol
4,dzire,2009,3358.208955,Petrol
...,...,...,...,...
85,etios g,2015,5895.522388,Petrol
86,corolla altis,2013,8208.955224,Petrol
87,corolla,2004,2238.805970,Petrol
88,corolla altis,2010,7835.820896,Petrol


In [28]:
# Log that you have started the Transform step
log2("Transform phase Started")

# Call the Transform function
transformed_data2 = transform2(extracted_data2)

# Log that you have completed the Transform step
log2("Transform phase Ended")
transformed_data2

,car_model,year_of_manufacture,price,fuel
0,alto 800,2017,4253.73,Petrol
1,ciaz,2015,10223.88,Diesel
2,ciaz,2015,11194.03,Petrol
3,ertiga,2015,9104.48,Petrol
4,dzire,2009,3358.21,Petrol
...,...,...,...,...
85,etios g,2015,5895.52,Petrol
86,corolla altis,2013,8208.96,Petrol
87,corolla,2004,2238.81,Petrol
88,corolla altis,2010,7835.82,Petrol


In [29]:

# Log that you have started the Load step
log2("Load phase Started")

# Call the Load function
load2(targetfile2,transformed_data2)

# Log that you have completed the Load step
log2("Load phase Ended")

# Log that you have completed the ETL process
log2("ETL Job Ended")

<details><summary>Click here for the solution</summary>

```

log("ETL Job Started")

log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")

log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

log("ETL Job Ended")
```

</details>


In [33]:
 with open("dealership_logfile.txt","r") as f:
        l = f.readlines()
        for i in l:
          print(i)

2022-Mar-18-21:50:14,ETL Job Started

2022-Mar-18-21:50:14,Extract phase Started

2022-Mar-18-21:50:14,Extract phase Ended

2022-Mar-18-21:50:14,Transform phase Started

2022-Mar-18-21:50:14,Transform phase Ended

2022-Mar-18-21:50:14,Load phase Started

2022-Mar-18-21:50:15,Load phase Ended

2022-Mar-18-21:50:15,ETL Job Ended



## Authors


Ramesh Sannareddy

Joseph Santarcangelo

Azim Hirjani


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2021-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork-23455645&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
